#Performance and benchmarking

<h1 id="toctitle">Contents</h1>
<ul id="toc"/>

---

Distinct but related concepts:

- Measuring
    - Benchmarking (how long does something take)
        - CPU
        - Memory (less so)
    - Profiling (which bits are slow)
        - CPU
        - ~~Memory~~ (not today)
- Optimizing

As with most programming jobs, a range of tools from simple to complex.

##Benchmarking

In approximate order of usefulness....

###Unix time
How long does our program take to run? On Linux/Mac we can do 

```
time somecommand
```

In iPython, prefix shell commands with `!`


In [1]:
!date

Wed Jul 29 10:27:51 BST 2015


Given output that looks like this:

```
real	0m0.490s 
user	0m0.457s 
sys	    0m0.032s 
```

- real is the wallclock time (affected by busy systems and other programs)
- user is the time spent executing our code
- sys is the time spent waiting for system calls (file IO, memory, network)

user+sys is probably the most useful. 

###Manual timing

Just measure the current time at the start of code, then again at end, and get the difference. 

`time.time()` gives us current UNIX epoch (number of seconds since midnight January 1st 1970 (don't ask.))

In [12]:
import time 
time.time()

1438162503.563577

On most systems this has very high resolution. 

In [6]:
import time 
start = time.time() 

# print the sum of the first million cube numbers
x = 0 
for i in range(1000000): 
    x = x + i ** 3 
print(x) 
 
end = time.time() 
print(end - start) 

249999500000250000000000
0.449609041214


This is arguably better than using the `time` command line as it doesn't include Python start up time, etc. However, still affected by other processes.

###`timeit` module

Python has a built in module for doing timing. From the command line:

In [14]:
!python -m timeit "4 ** 10"

10000000 loops, best of 3: 0.0292 usec per loop


Nice features:
- automatically runs the code many times to get an accurate measurement
- runs the whole thing three times and reports the best (accounts for other processes)
- gives the answer in easy to read units:

In [17]:
!python -m timeit "12345 in range(1000000)" 

10 loops, best of 3: 29.3 msec per loop


In the above code, do we spend more time constructing the range list or checking if the number is in it? Let's try just constructing the range:

In [20]:
!python -m timeit "range(1000000)" 

10 loops, best of 3: 31.2 msec per loop


Yep, takes loads to time to construct the list. Seperate that bit out with a setup (`-s`) command:

In [22]:
!python -m timeit -s "r=range(1000000)" "12345 in r" 

1000 loops, best of 3: 155 usec per loop


In iPython, we have magic convenience functions:

In [24]:
%timeit 4 ** 10

10000000 loops, best of 3: 34.6 ns per loop


In [25]:
r=range(1000000)
%timeit 12345 in r

1000 loops, best of 3: 154 µs per loop


In [27]:
%%timeit
# print the sum of the first million cube numbers
x = 0 
for i in range(1000000): 
    x = x + i ** 3 
print(x) 

249999500000250000000000
249999500000250000000000
249999500000250000000000
249999500000250000000000
1 loops, best of 3: 256 ms per loop


`timeit` is useful for quickly checking which approach is faster. `timeit` case study: which way is faster to calculate AT content - counting a and t, or looking at each base and keeping a tally?

In [65]:
from __future__ import division 

def at_count(dna): 
    return (dna.count('a') + dna.count('t')) / len(dna) 
 
def at_iter(dna): 
    a_count = 0 
    t_count = 0 
    for base in dna: 
        if base == 'a': 
            a_count = a_count + 1 
        elif base == 't': 
            t_count = t_count + 1 
    return (a_count + t_count) / len(dna) 

test_dna = 'atcgatcgatcatgatcggatcgtagctagcatctagtc' 
assert(at_count(test_dna) == at_iter(test_dna)) 

Which is faster?

In [42]:
%timeit at_count(test_dna)

The slowest run took 8.19 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 960 ns per loop


In [43]:
%timeit at_iter(test_dna)

100000 loops, best of 3: 6.78 µs per loop


Hmmm, something odd is going on. Short strings don't give reliable benchmarking results in Python due to optimizations in cPython. Let's try a more realistic input:

In [64]:
import random
def random_dna(length):
    return "".join([random.choice(['A','T','G','C']) for _ in range(length)])

In [73]:
random_dna(100)

'CGCGTTACTCTTGATGCCTAAGACTGGACTTTGTACCATGGTAGGGATCGCGAGCCGTGAAGTTTGGGGGGACCGCAAAGGCATGTGGACTTTCCTAGGT'

Now we can compare the two functions:

In [74]:
%timeit at_count(random_dna(10000))
%timeit at_iter(random_dna(10000))

100 loops, best of 3: 9.06 ms per loop
100 loops, best of 3: 9.41 ms per loop


Looks about equal, but wait: what if most of the time is spent generating the random DNA sequence? This is fairer:

In [75]:
d = random_dna(10000)
%timeit at_count(d)
%timeit at_iter(d)

10000 loops, best of 3: 26.2 µs per loop
1000 loops, best of 3: 1.47 ms per loop


Summary: 

- getting timing right is hard
- `count()` is faster than iteration (due to fast C code)

###Benchmarking memory

Here's the short story:

`pip install psutil`

then

In [79]:
import psutil, os 
 
process = psutil.Process(os.getpid()) 
mem = process.get_memory_info().rss / 1000 
print("Used this much memory: " + str(mem) + ' kb')

Used this much memory: 133734.4kb


Problem: this is useless in iPython notebooks as it includes everything that's been executed. For simple scripts, it's better. 

In [83]:
!python check_mem.py

Used this much memory: 40349 kb


This lets us investigate time/memory trade offs. We know that checking to see if a number is in a set is faster than checking to see if it's in a list:

In [89]:
l = range(1000000)
s = set(l)
%timeit 12345 in l
%timeit 12345 in s

1000 loops, best of 3: 154 µs per loop
The slowest run took 19.15 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 112 ns per loop


but how much longer does it take to create the data structure in the first place?

In [90]:
%timeit list(range(1000000))
%timeit set(range(1000000))

10 loops, best of 3: 56 ms per loop
10 loops, best of 3: 113 ms per loop


and how much more memory does it take to hold the set?

In [92]:
!python list_mem.py
!python set_mem.py

Used this much memory: 40349 kb
Used this much memory: 65953 kb


Conclusions:
- if we need to create a list once then check membership many times, a set will be faster
- if we need to create many lists, a set might be slower
- a set will use more (x1.5) memory for these ranges 

Of course, everything might be different for non-integers!

##Profiling

Profiling is the process of taking an existing piece of code and identifying which bits are taking the time. 

Scenario: given

- a single long DNA sequence
- a collection of interesting 4-base motifs

we want to identify frequently-occuring (say 50 times) 4-base motifs in the sequence and divide them into ones that are also on the interesting list, and ones that aren't. 



In [98]:
# create a random dna sequence
dna = random_dna(10000)

# create 100 random interesting motifs
motifs = [random_dna(4) for _ in range(100)]

In [102]:
%%timeit
# standard kmer counting code to identify frequent chunks
frequent_chunks = [] 
for start in range(len(dna) - 3): 
    chunk = dna[start:start + 4] 
    if dna.count(chunk) > 50: 
        frequent_chunks.append(chunk) 

# now check each chunk to see if it's in the list of motifs
for chunk in frequent_chunks: 
    if chunk in motifs: 
        print(chunk + " is frequent and interesting") 
    else: 
        print(chunk + " is frequent but not interesting")

TCCA is frequent and interesting
CCTT is frequent but not interesting
TCCA is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
TATA is frequent but not interesting
TATA is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent but not interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
TCCA is frequent and interesting
TGTT is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent but not interest

How can we speed this program up? We know that checking to see if an element is in a list is slow, so let's change it to a set:

In [103]:
# create 100 random interesting motifs
motifs = set([random_dna(4) for _ in range(100)])

In [104]:
%%timeit
# standard kmer counting code to identify frequent chunks
frequent_chunks = [] 
for start in range(len(dna) - 3): 
    chunk = dna[start:start + 4] 
    if dna.count(chunk) > 50: 
        frequent_chunks.append(chunk) 

# now check each chunk to see if it's in the list of motifs
for chunk in frequent_chunks: 
    if chunk in motifs: 
        print(chunk + " is frequent and interesting") 
    else: 
        print(chunk + " is frequent but not interesting")

TCCA is frequent and interesting
CCTT is frequent but not interesting
TCCA is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
TATA is frequent but not interesting
TATA is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
TCCA is frequent and interesting
TGTT is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
TATA

Why did this not work? Probably because the line

```python
    if chunk in motifs: 
```

doesn't actually get execute that often:

In [106]:
len(frequent_chunks)

375

What we need is a way to see which lines are taking up the most time.

###Profiling with cProfile

`cProfile` is a built in module for profiling functions. It's easy to use:

In [107]:
import cProfile 

# we have to turn the code into a function so we can pass its name to run()
def classify_chunks():
    frequent_chunks = [] 
    for start in range(len(dna) - 3): 
        chunk = dna[start:start + 4] 
        if dna.count(chunk) > 50: 
            frequent_chunks.append(chunk) 

    for chunk in frequent_chunks: 
        if chunk in motifs: 
            print(chunk + " is frequent and interesting") 
        else: 
            print(chunk + " is frequent but not interesting")

cProfile.run("classify_chunks()")

TCCA is frequent and interesting
CCTT is frequent but not interesting
TCCA is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
TATA is frequent but not interesting
TATA is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
TCCA is frequent and interesting
TGTT is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
TATA

cProfile gives us tabular output

- ncalls, which tells us how many times the function was called
- tottime, which tells us the total amount of time that was spent in that function (not including sub functions)
- percall, which tells us the amount of time that was spent in that function (not including sub functions) each time it was called
- cumtime, which is like tottime but does include sub functions
- another percall, which is like the first one except that it does include sub functions
- filename, which tells us the filename, line number, and name of the function or method


What can we see from the cProfile output?

- the code took about 0.6 seconds to run (slightly longer than before due to overhead)
- 17877 functions were called

Let's look at some specific functions:

`classify_chunks()`
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
     1  0.009    0.009    0.620    0.620   time_profile.py:13(classify_chunks) 
```
- was called once
- very small tottime i.e. not much going on 
- but very big cumtime, so most of time in functions called inside it

`append()` method of lists
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
   419  0.000    0.000    0.000    0.000   {method 'append' of 'list' objects} 
```
- called 419 times (once for each frequent chunk)
- but tottime and cumtime so low they're rounded to zero

`count()` method of strings
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
  9997  0.612    0.000    0.612    0.000   {method 'count' of 'str' objects} 
```
- called around 10000 times (once per chunk in the DNA sequence)
- percall time is very low - counting is fasta
- but tottime is big - most of the time of the program. 

cProfile only measures function calls, i.e. not stuff like

```python
dna[start:start + 4] 
dna.count(chunk) > 50
chunk in motifs
```
To use it well, we need structured code e.g. if we split our code into two functions:

In [108]:
def get_frequent_chunks(dna): 
    frequent_chunks = [] 
    for start in range(len(dna) - 3): 
        chunk = dna[start:start + 4] 
        if dna.count(chunk) > 50: 
            frequent_chunks.append(chunk) 
    return frequent_chunks 
 
def print_chunks(chunks): 
    for chunk in chunks: 
        if chunk in motifs: 
            print(chunk + " is frequent and interesting") 
        else: 
            print(chunk + " is frequent but not interesting") 
            
def classify_chunks(): 
    frequent_chunks = get_frequent_chunks(dna) 
    print_chunks(frequent_chunks) 

cProfile.run("classify_chunks()") 

TCCA is frequent and interesting
CCTT is frequent but not interesting
TCCA is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
ATAA is frequent but not interesting
TATA is frequent but not interesting
TATA is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
ATAA is frequent but not interesting
TCCA is frequent and interesting
CCAG is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
CCTT is frequent but not interesting
TGTT is frequent but not interesting
TCCA is frequent and interesting
TGTT is frequent but not interesting
CCTT is frequent but not interesting
ATAT is frequent and interesting
TATA

We get more useful output:

```
ncalls tottime percall   cumtime   percall filename:lineno(function) 
 1    0.005    0.005      0.619     0.619  time_profile.py:14(get_frequent_chunks)
 1    0.004    0.004      0.004     0.004  time_profile.py:22(print_chunks) 
```

Most of the time is counting the chunks, not classifying them (hence why our speed-up idea didn't work)

###Profiling with line_profiler

`line_profiler` is a third part module that you have to install separately:

`pip install line_profiler`

It measures execution time per line. To use it we add a `@profile` decorator to the function and run from the command line with:

`kernprof -l -v slowfast.py`

Let's try it:

In [111]:
!kernprof -l -v chunks.py

ATCG is frequent but not interesting
GATT is frequent and interesting
ACGA is frequent but not interesting
ATCG is frequent but not interesting
GCGC is frequent and interesting
ACGA is frequent but not interesting
GTAA is frequent but not interesting
GTAA is frequent but not interesting
GATT is frequent and interesting
GGCG is frequent but not interesting
GCGC is frequent and interesting
GATT is frequent and interesting
GATT is frequent and interesting
ACGA is frequent but not interesting
GATT is frequent and interesting
ATCG is frequent but not interesting
GGCG is frequent but not interesting
GGCG is frequent but not interesting
GATT is frequent and interesting
ACGA is frequent but not interesting
GTAA is frequent but not interesting
GGCG is frequent but not interesting
GTAA is frequent but not interesting
GTAA is frequent but not interesting
GGCG is frequent but not interesting
GCGC is frequent and interesting
GATT is frequent and interesting
GGCG is frequent but not interesting
GATT

Hopefully the output shows the usefulness - we can see immediately that 

`if dna.count(chunk) > 50:`

uses 96% of the time. Let's switch to a dict and keep a tally:

In [112]:
def classify_chunks(): 
    chunk_count = {}
    for start in range(len(dna) - 3):
        chunk = dna[start:start + 4]
        current_count = chunk_count.get(chunk, 0)
        new_count = current_count + 1 
        chunk_count[chunk] = new_count

    for chunk, count in chunk_count.items():
        if count > 50: 
            if chunk in motifs:
                print(chunk + " is frequent and interesting")
            else:
                print(chunk + " is frequent but not interesting")

In [113]:
!kernprof -l -v chunks.py

CCAT is frequent and interesting
CCAA is frequent but not interesting
TGTT is frequent but not interesting
TTCC is frequent but not interesting
TCAT is frequent but not interesting
GTTC is frequent and interesting
CCCC is frequent but not interesting
CATG is frequent but not interesting
CGCT is frequent but not interesting
Wrote profile results to chunks.py.lprof
Timer unit: 1e-06 s

Total time: 0.056549 s
File: chunks.py
Function: classify_chunks at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           @profile
    12                                           def classify_chunks(): 
    13         1            2      2.0      0.0      chunk_count = {}
    14      9998         9893      1.0     17.5      for start in range(len(dna) - 3):
    15      9997        11790      1.2     20.8          chunk = dna[start:start + 4]
    16      9997        12889      1.3     22.8          current_count = chunk_count.get(chunk, 0)


The improvement is dramatic. We could go on from here to make further improvements. 

###Profiling with realistic inputs

For many programs the data greatly affects the profiling results. Here are functions to calculate AT content and to check if two sequences share the same first five bases:

In [114]:
def at_content(dna): 
    return (dna.count('A') + dna.count('T')) / len(dna) 

def same_start(dna1, dna2): 
    return dna1[0:5] == dna2[0:5]

We want to take a large collection of DNA sequences and identify those that

- have an AT content greater than some cutoff value and
- have the same first five bases as at least on other sequence

Here's a first attempt:

In [116]:
def find_interesting(dnas, cutoff):
    interesting = set()
    for one in dnas:
        at = at_content(one)
        if at > cutoff:
            for two in dnas:
                if one != two and same_start(one, two):
                    interesting.add(one)

    return(interesting)

Which function takes the most time? Let's send it though `line_profiler` with 1000 sequences of 1000 bases:

In [118]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.023879 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1            4      4.0      0.0      interesting = set()
    19      1001          679      0.7      2.8      for one in dnas:
    20      1000        14497     14.5     60.7          at = at_content(one)
    21      1000          811      0.8      3.4          if at > cutoff:
    22      3003         2252      0.7      9.4              for two in dnas:
    23      3000         5620      1.9     23.5                  if one != two and same_start(one, two):
    24         5           16      3.2      0.1                      interesting.add(one)
    25                                           
    26         1            0      0.0   

The calls to `at_content()` take most of the time. But, if we try a bigger dataset with 10,000 sequences:

In [119]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.661217 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1           11     11.0      0.0      interesting = set()
    19     10001         6547      0.7      1.0      for one in dnas:
    20     10000       142592     14.3     21.6          at = at_content(one)
    21     10000         8209      0.8      1.2          if at > cutoff:
    22    190019       146992      0.8     22.2              for two in dnas:
    23    190000       356592      1.9     53.9                  if one != two and same_start(one, two):
    24       168          273      1.6      0.0                      interesting.add(one)
    25                                           
    26         1            1      1.0   

Now `same_start()` is taking the most time. Why? because it's in the inner loop, so the number of calls scales with the square of the size of the dataset - O(N) vs O(N<sup>2</sup>).

What happens if we keep the dataset size the same at 1000 sequences, but decrease the AT content cutoff so that more sequences pass it (0.525):

In [120]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.126483 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1            4      4.0      0.0      interesting = set()
    19      1001          545      0.5      0.4      for one in dnas:
    20      1000        12943     12.9     10.2          at = at_content(one)
    21      1000          728      0.7      0.6          if at > cutoff:
    22     51051        30503      0.6     24.1              for two in dnas:
    23     51000        81615      1.6     64.5                  if one != two and same_start(one, two):
    24        56          144      2.6      0.1                      interesting.add(one)
    25                                           
    26         1            1      1.0   

`same_start()` dominates again. In both these cases the code is unchanged, but by changing the dataset we get opposite results about which function to optimize first. 

**always carry out benchmarking and profiling on realistic data!**

###Performance optimization tips

- File and network IO is slow, so minimize it
- Existing modules are likely to be fast, so use them (scipy/numpy/pandas)
- Avoid calculating the same thing multiple times
- Functional structures (maps/comprehensions) tend to be faster than loops
- Pick data structures with the properties you want (list vs. set)

and some advanced things to know about:

- write inline code in faster languages
- use parallel code
- let a database do the heavy lifting if you can

##Exercises

See how much faster you can make the chunks function from above. What effect do the various parameters (chunk length, DNA sequence length, interesting motif count, frequency cutoff) have on the execution time? what effect do they have on the distribution of execution time across lines?

Do the same with one of your solutions to a previous exercise, or some of your own code. 


